#Tensorflow COVID-19 Classification by Chest X-Ray

Import necessary module and package.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

from keras.models import load_model

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/tensorflow_core/python/pywrap_tensorflow_internal.py:15: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
 import imp

Import and initiate MLFlow and AML package.

In [5]:
import mlflow
import mlflow.azureml
import azureml.mlflow
import azureml.core

from azureml.core import Workspace
from azureml.mlflow import get_portal_url

subscription_id = 'PLEASE-ENTER-YOUR-OWN-AZURE-SUBSCRIPTION-ID'

# Azure Machine Learning resource group NOT the managed resource group
resource_group = 'PLEASE-ENTER-YOUR-OWN-RESOURCES-GROUP-IN-AZURE' 

#Azure Machine Learning workspace name, NOT Azure Databricks workspace
workspace_name = 'PLEASE-ENTER-YOUR-OWN-AML-WORKSPACE-NAME'  

# Instantiate Azure Machine Learning workspace
ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group)

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.

Setup MLFlow & AML experiment.

In [7]:
uri = ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(uri)
experiment_name = 'tensorflow_covid19_classification_adb'
mlflow.set_experiment(experiment_name)

INFO: 'tensorflow_covid19_classification_adb' does not exist. Creating a new experiment

Print currently install version of Tensorflow and Keras.

In [9]:
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)

2.1.0
2.3.1

Mount Azure Blob Storage to obtain data.

In [11]:
dbutils.fs.mount(
  source = "wasbs://PLEASE-ENTER-YOUR-OWN-BLOB-STORAGE-ACCOUNT-NAME.blob.core.windows.net",
  mount_point = "/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT",
  extra_configs = {"fs.azure.account.key.PLEASE-ENTER-YOUR-OWN-BLOB-STORAGE-ACCOUNT-NAME.blob.core.windows.net":dbutils.secrets.get(scope = "secret01", key = "key01")})

Out[3]: True

Unmount Azure Blob Storage as needed.

In [13]:
dbutils.fs.unmount("/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT")

/mnt/XXXXX has been unmounted.
Out[2]: True

Show currently mounted dbfs via fs command.

In [15]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/MLSample01/,MLSample01/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/mlflow/,mlflow/,0
dbfs:/mnt/,mnt/,0
dbfs:/tmp/,tmp/,0


Copy util.py to Databricks driver directory.

In [17]:
dbutils.fs.cp("/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/util.py", "/databricks/driver/util/util.py")
display(dbutils.fs.ls("/databricks/driver/util"))

path,name,size
dbfs:/databricks/driver/util/__pycache__/,__pycache__/,0
dbfs:/databricks/driver/util/util.py,util.py,3613


Import util module.

In [19]:
import sys
sys.path.append('/dbfs/databricks/driver/util/')
import util

Read the data

In [21]:
train_df = pd.read_csv("/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/data/train.csv")
valid_df = pd.read_csv("/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/data/valid.csv")

test_df = pd.read_csv("/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/data/test.csv")

train_df.head()

,Image,COVID19,Normal
0,COVID-19 (1).png,1,0
1,COVID-19 (10).png,1,0
2,COVID-19 (100).png,1,0
3,COVID-19 (101).png,1,0
4,COVID-19 (102).png,1,0


Define the labels.

In [23]:
labels = ['COVID19', 'Normal']

Preparing images
* Using ImageDataGenerator class from Keras to build a "generator" for images specified in a dataframe.
* Perform basic data augmentation such as random horizontal flipping of images.
* Transform the values in each batch so that their mean is 0 and their standard deviation is 1.
* Converts single channel X-ray images (gray-scale) to a three-channel format by repeating the values in the image across all channels.

In [25]:
def get_train_generator(df, image_dir, x_col, y_cols, shuffle=True, batch_size=4, seed=1, target_w = 320, target_h = 320):
    print("getting train generator...") 
    # normalize images
    image_generator = ImageDataGenerator(
        samplewise_center=True,
        samplewise_std_normalization= True)
    
    # flow from directory with specified batch size
    # and target image size
    generator = image_generator.flow_from_dataframe(
            dataframe=df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=shuffle,
            seed=seed,
            target_size=(target_w,target_h))
    
    return generator

Build a separated generator for valid and test sets.

In [27]:
def get_test_and_valid_generator(valid_df, test_df, train_df, image_dir, x_col, y_cols, sample_size=100, batch_size=4, seed=1, target_w = 320, target_h = 320):
    print("getting train and valid generators...")
    # get generator to sample dataset
    raw_train_generator = ImageDataGenerator().flow_from_dataframe(
        dataframe=train_df, 
        directory=IMAGE_DIR, 
        x_col="Image", 
        y_col=labels, 
        class_mode="raw", 
        batch_size=sample_size, 
        shuffle=True, 
        target_size=(target_w, target_h))
    
    # get data sample
    batch = raw_train_generator.next()
    data_sample = batch[0]

    # use sample to fit mean and std for test set generator
    image_generator = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization= True)
    
    # fit generator to sample from training data
    image_generator.fit(data_sample)

    # get test generator
    valid_generator = image_generator.flow_from_dataframe(
            dataframe=valid_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))

    test_generator = image_generator.flow_from_dataframe(
            dataframe=test_df,
            directory=image_dir,
            x_col=x_col,
            y_col=y_cols,
            class_mode="raw",
            batch_size=batch_size,
            shuffle=False,
            seed=seed,
            target_size=(target_w,target_h))
    return valid_generator, test_generator

Run generator for training data and one each of test and validation datasets.

In [29]:
IMAGE_DIR = "/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/images/"
train_generator = get_train_generator(train_df, IMAGE_DIR, "Image", labels)
valid_generator, test_generator= get_test_and_valid_generator(valid_df, test_df, train_df, IMAGE_DIR, "Image", labels)

getting train generator...
Found 353 validated image filenames.
getting train and valid generators...
Found 353 validated image filenames.
Found 133 validated image filenames.
Found 43 validated image filenames.

Picking up one of training sample to visualize.

In [31]:
x, y = train_generator.__getitem__(0)
plt.imshow(x[0]);

Checking imbalance class situation in datasets by plot the frequency of each labels in dataset.

In [33]:
plt.xticks(rotation=90)
plt.bar(x=labels, height=np.mean(train_generator.labels, axis=0))
plt.title("Frequency of Each Class")
plt.show()

Function to calculate these frequences for each label in dataset.

In [35]:
def compute_class_freqs(labels):
    
    # total number of patients (rows)
    N = labels.shape[0]
    
    positive_frequencies = np.sum(labels, axis=0) / N
    negative_frequencies = 1 - positive_frequencies

    return positive_frequencies, negative_frequencies

Loss function that calculates the weighted loss for each batch.

In [37]:
def get_weighted_loss(pos_weights, neg_weights, epsilon=1e-7):
    def weighted_loss(y_true, y_pred):
        # initialize loss to zero
        loss = 0.0
        
        for i in range(len(pos_weights)):
            # for each class, add average weighted loss for that class
            loss_pos = -1 * K.mean(pos_weights[i] * y_true[:, i] * K.log(y_pred[:, i] + epsilon))
            loss_neg = -1 * K.mean(neg_weights[i] * (1 - y_true[:, i]) * K.log(1 - y_pred[:, i] + epsilon))
            loss += loss_pos + loss_neg
        
        return loss

    return weighted_loss

Compute frequencies for training data.

In [39]:
freq_pos, freq_neg = compute_class_freqs(train_generator.labels)
freq_pos

Out[15]: array([0.43342776, 0.56657224])

Visualize these two contribution ratios of COVID19 and Normal.

In [41]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": freq_pos})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} for l,v in enumerate(freq_neg)], ignore_index=True)
plt.xticks(rotation=90)
f = sns.barplot(x="Class", y="Value", hue="Label" ,data=data)

Re-balancing the contribution of positive and negative labels.

In [43]:
pos_weights = freq_neg
neg_weights = freq_pos
pos_contribution = freq_pos * pos_weights 
neg_contribution = freq_neg * neg_weights

Verify again by plotting the two contributions next to each other again.

In [45]:
data = pd.DataFrame({"Class": labels, "Label": "Positive", "Value": pos_contribution})
data = data.append([{"Class": labels[l], "Label": "Negative", "Value": v} 
                        for l,v in enumerate(neg_contribution)], ignore_index=True)
plt.xticks(rotation=90)
sns.barplot(x="Class", y="Value", hue="Label" ,data=data);

Load pre-trained DenseNet121 model and then add two layers on top of it.

In [47]:
# create the base pre-trained model
base_model = DenseNet121(weights='/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/models/densenet.hdf5', include_top=False)

x = base_model.output

# add a global spatial average pooling layer
x = GlobalAveragePooling2D()(x)

# and a logistic layer
predictions = Dense(len(labels), activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss=get_weighted_loss(pos_weights, neg_weights), metrics=['accuracy'])

Start the model training.

In [49]:
history = model.fit_generator(train_generator, 
                              validation_data=valid_generator,
                              steps_per_epoch=100, 
                              validation_steps=25, 
                              epochs = 3)

Epoch 1/3
 1/100 [..............................] - ETA: 1:31:09 - loss: 0.7286 - accuracy: 0.5000 2/100 [..............................] - ETA: 47:23 - loss: 0.4874 - accuracy: 0.7500  3/100 [..............................] - ETA: 32:40 - loss: 0.3665 - accuracy: 0.8333 4/100 [>.............................] - ETA: 25:17 - loss: 0.5763 - accuracy: 0.8125 5/100 [>.............................] - ETA: 20:50 - loss: 0.4666 - accuracy: 0.8500 6/100 [>.............................] - ETA: 17:51 - loss: 0.4453 - accuracy: 0.8750 7/100 [=>............................] - ETA: 15:44 - loss: 0.3864 - accuracy: 0.8929 8/100 [=>............................] - ETA: 14:07 - loss: 0.3468 - accuracy: 0.9062 9/100 [=>............................] - ETA: 12:51 - loss: 0.3157 - accuracy: 0.9167 10/100 [==>...........................] - ETA: 11:50 - loss: 0.2859 - accuracy: 0.9250 11/100 [==>...........................] - ETA: 11:00 - loss: 0.2613 - accuracy: 0.9318 12/100 [==>...........................] - ETA: 10:17 - loss: 0.3721 - accuracy: 0.9167 13/100 [==>...........................] - ETA: 9:41 - loss: 0.3473 - accuracy: 0.9231  14/100 [===>..........................] - ETA: 9:09 - loss: 0.3290 - accuracy: 0.9286 15/100 [===>..........................] - ETA: 8:41 - loss: 0.3234 - accuracy: 0.9167 16/100 [===>..........................] - ETA: 8:17 - loss: 0.3347 - accuracy: 0.9062 17/100 [====>.........................] - ETA: 7:55 - loss: 0.3325 - accuracy: 0.8971 18/100 [====>.........................] - ETA: 7:35 - loss: 0.3207 - accuracy: 0.9028 19/100 [====>.........................] - ETA: 7:17 - loss: 0.3097 - accuracy: 0.9079 20/100 [=====>........................] - ETA: 7:01 - loss: 0.2956 - accuracy: 0.9125 21/100 [=====>........................] - ETA: 6:45 - loss: 0.2879 - accuracy: 0.9167 22/100 [=====>........................] - ETA: 6:31 - loss: 0.2748 - accuracy: 0.9205 23/100 [=====>........................] - ETA: 6:18 - loss: 0.2637 - accuracy: 0.9239

Plotting model training summary.

In [51]:
# setting up experiment name in AML
experiment_name = 'tensorflow_covid19_classification_adb'
mlflow.set_experiment(experiment_name)
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

with mlflow.start_run():
  mlflow.log_param('epochs', 3)
  mlflow.log_param('batch_size', 4)

  lol = history.history['loss']
  for x in range(len(lol)):
      mlflow.log_metric('loss', np.float(lol[x]), step=x),
      loa = history.history['accuracy']
  for x in range(len(loa)):
      mlflow.log_metric('accuracy', np.float(loa[x]), step=x),
  mlflow.end_run()

Save the trained model in h5 format.

In [53]:
model.save('/tmp/my_model.h5')
dbfs_model_path = 'dbfs:/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/models/my_model.h5'
dbutils.fs.cp('file:/tmp/my_model.h5', dbfs_model_path)

Out[42]: True

Load the trained model as need.

In [55]:
#model.load_weights("/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/models/my_model.h5")

Use predict_generator function to generate the prediction for the images in our test set.

In [57]:
predicted_vals = model.predict_generator(test_generator, steps = len(test_generator))

Generate ROC curve.

In [59]:
auc_rocs = util.get_roc_curve(labels, predicted_vals, test_generator)

Load the data labels from data.

In [61]:
df = pd.read_csv("/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/data/train.csv")
IMAGE_DIR = "/dbfs/mnt/PLEASE-ENTER-YOUR-OWN-MOUNT-POINT/images/"

# only show the labels with top 4 AUC
labels_to_show = np.take(labels, np.argsort(auc_rocs)[::-1])[:4]

Load test image and generate Gradcam (COVID19 Positive).

In [63]:
util.compute_gradcam(model, 'COVID-19(208).png', IMAGE_DIR, df, labels, labels_to_show)

Load test image and generate Gradcam (Normal).

In [65]:
util.compute_gradcam(model, 'NORMAL (941).png', IMAGE_DIR, df, labels, labels_to_show)